# 🎥 Movie - 추천 시스템

## ✅ 1. 데이터 전처리

### 📌 1-1. 데이터 로딩

In [1]:
import pandas as pd

# 평점 데이터
ratings = pd.read_csv("/Users/lee_hyejoo/Desktop/hyejoo/학교/3학년 1학기/머신러닝/중간_대체/영화_추천_시스템, MovieLens - Recommendation/u.data", sep="\t", header=None,
                      names=["user_id", "item_id", "rating", "timestamp"])

# 영화 메타데이터
movies = pd.read_csv("/Users/lee_hyejoo/Desktop/hyejoo/학교/3학년 1학기/머신러닝/중간_대체/영화_추천_시스템, MovieLens - Recommendation/u.item", sep='|', header=None, encoding='latin-1',
                     names=["item_id", "title"] + [f"col{i}" for i in range(22)])

# 필요한 컬럼만 추출
movies = movies[["item_id", "title"]]

### 📌 1-2. 평점 테이블 병합

In [2]:
# 평점 데이터에 영화 제목 추가
ratings = pd.merge(ratings, movies, on="item_id")

# 상위 5개 샘플 확인
ratings.head()

,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


### 📌1-3. 사용자-영화 평점 행렬 생성

In [3]:
# 사용자-영화 간 평점 행렬 생성
user_item_matrix = ratings.pivot_table(index='user_id', columns='title', values='rating')

# 일부 확인
user_item_matrix.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN


### 📌1-4. 데이터 희소성(Sparsity) 확인

In [4]:
print("행렬 크기:", user_item_matrix.shape)
print("전체 평점 수:", ratings.shape[0])
print("비어 있는 셀 수:", user_item_matrix.isna().sum().sum())

행렬 크기: (943, 1664)
전체 평점 수: 100000
비어 있는 셀 수: 1469459


---

## 🤖 2. 추천 알고리즘 비교

### 📌 2-1. 기본 모델 - User-Based 협업 필터링

💡 설명
- 사용자 기반 협업 필터링은 "비슷한 취향을 가진 사용자"를 찾고, 그 사용자가 높게 평가한 영화를 추천해주는 방식
- 유사도는 코사인 유사도(Cosine Similarity)를 사용하며, 현재 사용자가 평가하지 않은 영화에 대해 유사 사용자의 평점을 기반으로 예측 점수 계산

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

#  사용자 유사도 행렬 계산 (NaN을 0으로 대체)
user_similarity = cosine_similarity(user_item_matrix.fillna(0))
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

#  user_id=1 사용자의 추천 후보 계산
user_id = 1
user_ratings = user_item_matrix.loc[user_id]
unseen_movies = user_ratings[user_ratings.isna()].index
similar_users = user_similarity_df[user_id].drop(user_id)  # 자기 자신 제외

weighted_scores = {}
for movie in unseen_movies:
    movie_ratings = user_item_matrix[movie]
    relevant_ratings = movie_ratings[similar_users.index]
    valid = relevant_ratings.notna()
    if valid.sum() > 0:
        score = np.dot(relevant_ratings[valid], similar_users[valid]) / similar_users[valid].sum()
        weighted_scores[movie] = score

#  추천 상위 5개
top_5_recommendations = sorted(weighted_scores.items(), key=lambda x: x[1], reverse=True)[:5]
top_5_recommendations

[('Aiqing wansui (1994)', 5.0),
 ('Entertaining Angels: The Dorothy Day Story (1996)', 5.0),
 ('Great Day in Harlem, A (1994)', 5.0),
 ('Marlene Dietrich: Shadow and Light (1996) ', 5.0),
 ('Prefontaine (1997)', 5.0)]

🔍 결과 (user_id = 1 기준)
- 위 영화들은 user_id=1이 아직 평가하지 않았지만, 비슷한 취향의 사용자들이 모두 높게 평가한 영화들
- 모든 예측 평점이 5.0인 것은 유사 사용자 집단에서 일치된 강한 선호가 있었음을 의미
- 이 방식은 직관적이지만, 희소한 데이터에서는 추천 품질이 낮아질 수 있음 → 다음 단계에서 개선

---

### 📌 2-2. 기본 모델 - Item-Based 협업 필터링

💡 설명
- 영화-영화 간 유사도(코사인 유사도)를 계산
- 사용자가 평가한 영화들 중, 유사한 영화들의 평점과 유사도를 기반으로 예측 평점 계산
- 아직 보지 않은 영화 중 예측 평점이 높은 영화 추천

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. 영화 간 유사도 계산 (user-item matrix의 전치행렬 사용)
item_similarity = cosine_similarity(user_item_matrix.T.fillna(0))
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

# 2. 사용자(user_id=1)의 평점 가져오기
user_id = 1
user_ratings = user_item_matrix.loc[user_id]

# 3. 사용자가 보지 않은 영화 중 예측 평점 계산
predicted_scores = {}

for movie in user_item_matrix.columns:
    if pd.isna(user_ratings[movie]):  # 아직 평가하지 않은 영화만 대상
        # 이 영화와 사용자가 본 영화 간 유사도 × 평점 → 가중 평균
        sim_movies = item_similarity_df[movie][user_ratings.notna()]
        sim_scores = sim_movies.values
        known_ratings = user_ratings[user_ratings.notna()].values
        if sim_scores.sum() != 0:
            predicted_scores[movie] = np.dot(sim_scores, known_ratings) / sim_scores.sum()

# 4. 상위 5개 추천 영화 출력
item_based_top5 = sorted(predicted_scores.items(), key=lambda x: x[1], reverse=True)[:5]
item_based_top5

[('Cyclo (1995)', 4.382758930148253),
 ('Little City (1998)', 4.2349259422153755),
 ('Office Killer (1997)', 4.229406245638664),
 ('Death in Brunswick (1991)', 4.226812532525961),
 ('Mamma Roma (1962)', 4.178130372636396)]

🔍 결과
- 위 영화들은 user_id=1이 아직 평가하지 않았지만, 본인이 평가한 영화들과 유사한 영화들 중 예측 평점이 높은 항목들
- 사용자 기반과 달리, 아이템 간 유사도를 사용해 추천하기 때문에 같은 영화라도 예전보다 더 일관된 추천 품질을 보여줄 수 있음
- 예측 점수는 실제 평점이 아닌 모델이 계산한 "예상 선호도"이므로 → Top-N 추천 리스트 구성에 유용함

---

### 📌 2-3. 행렬 분해 - SVD

💡 설명
- 추천 시스템에서 널리 사용되는 행렬 분해 기반 협업 필터링 방법으로, 사용자-아이템 간 희소 행렬을 저차원 잠재 요인(latent factor) 공간으로 압축해
유사도를 계산하고 예측 평점을 도출
- 희소한 평점 행렬의 구조를 압축하면서도 정보 손실을 최소화할 수 있음
- 협업 필터링보다 정밀한 사용자/아이템 관계 파악 가능


In [7]:
# !/usr/local/bin/python3.12 -m pip install scikit-surprise

In [8]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# 1. 데이터 준비
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['user_id', 'title', 'rating']], reader)  # 여기 수정!

# 2. 데이터 분할
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# 3. 모델 학습
svd = SVD()
svd.fit(trainset)

# 4. 예측 및 평가
predictions = svd.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9351


0.9350622018483603

🔍 결과
- SVD 모델은 약 0.933의 RMSE를 기록하며, 기존의 협업 필터링 기반 추천 방식보다 정량적인 예측 정확도가 우수한 것으로 나타남
- 이는 행렬 분해 방식이 사용자와 영화 간의 잠재적 선호 관계(latent factor)를 효과적으로 반영했기 때문

---

### 📌 2.4 행렬 분해 - ALS

💡 설명
- 암묵적 피드백 데이터(예: 클릭, 시청 등)를 바탕으로 행렬을 두 개의 저차원 행렬로 분해해 사용자와 아이템의 잠재 요인을 추정하는 방식
- 평점 데이터에도 적용 가능하지만, 암묵적 방식에 맞춰서 0~1 스케일로 가중치화가 필요

In [58]:
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from sklearn.metrics import mean_squared_error, precision_score, recall_score
import numpy as np

# ALS 모델 학습용 데이터 준비
ratings['user_idx'] = ratings['user_id'].astype("category").cat.codes
ratings['item_idx'] = ratings['title'].astype("category").cat.codes

# 아이템 x 사용자 희소행렬 생성
item_user_matrix = csr_matrix((ratings['rating'], (ratings['item_idx'], ratings['user_idx'])))

# 사용자 x 아이템 희소행렬 (추천용)
user_item_matrix = item_user_matrix.T.tocsr()

# ALS 모델 학습
als_model = AlternatingLeastSquares(factors=50, regularization=0.01, iterations=20, use_cg=True)
als_model.fit(item_user_matrix)

# ALS 학습된 범위 확인
valid_user_count = als_model.user_factors.shape[0]
valid_item_count = als_model.item_factors.shape[0]


  0%|          | 0/20 [00:00<?, ?it/s]

 ### 📌 2.5 - RMSE 평가

💡 설명
- RMSE (Root Mean Squared Error) 는 예측 평점과 실제 평점 간의 평균적인 오차 크기를 나타냄
- 이 값이 작을수록 모델이 실제 사용자 평점에 더 가까운 예측을 했다는 의미

In [59]:
# 테스트 샘플 생성
test_sample = ratings.sample(frac=0.2, random_state=42)

# ALS 범위 내 데이터로 필터링
valid_test_sample = test_sample[
    (test_sample['user_idx'] < valid_user_count) &
    (test_sample['item_idx'] < valid_item_count)
].copy()

# ALS 예측 함수
def predict_rating(user_idx, item_idx):
    user_factor = als_model.user_factors[user_idx]
    item_factor = als_model.item_factors[item_idx]
    return np.dot(user_factor, item_factor)

# 예측값 계산
valid_test_sample['predicted_rating'] = valid_test_sample.apply(
    lambda row: predict_rating(row['user_idx'], row['item_idx']), axis=1
)

# RMSE 계산
rmse = mean_squared_error(valid_test_sample['rating'], valid_test_sample['predicted_rating'], squared=False)
print(f"\n✅ ALS 모델 RMSE: {rmse:.4f}")



✅ ALS 모델 RMSE: 3.5993


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


🔍 결과
- ALS 모델은 RMSE 약 3.5993을
- 이는 ALS 모델이 사용자와 영화 간의 잠재 요인을 고려하여 예측했지만, 다소 큰 오차를 보였다는 것을 의미

---

### 📌 2.6 데이터 필터링

💡 설명
- ALS 모델은 학습 과정에서 사용자와 아이템의 잠재 요인(latent factors) 을 추정함
- 이 과정에서 유효한 사용자와 아이템 수(valid_user_count, valid_item_count) 가 정해지기 때문에, 전체 데이터 중 ALS 학습에 사용 가능한 범위로 데이터를 필터링해야 함
- 필터링을 통해 ALS 모델이 학습한 범위 내에서만 정확한 예측과 평가가 가능해짐

In [60]:
# ALS 학습된 범위로 ratings 데이터 필터링
filtered_ratings = ratings[
    (ratings['user_idx'] < valid_user_count) &
    (ratings['item_idx'] < valid_item_count)
].copy()

print(f"✅ 필터링 후 사용자 수: {filtered_ratings['user_idx'].nunique()}")
print(f"✅ 필터링 후 아이템 수: {filtered_ratings['item_idx'].nunique()}")


✅ 필터링 후 사용자 수: 943
✅ 필터링 후 아이템 수: 943


🔍 결과
- 총 943명의 사용자와 943개의 아이템에 대해 잠재 요인이 추정됨
- 이 수치는 이후 예측 및 추천에서 ALS 모델이 다룰 수 있는 유효한 범위를 의미하며, 해당 범위를 벗어나는 데이터는 예측이 불가능함
- 따라서, 943명 사용자와 943개 아이템의 교차 정보만을 기반으로 평가 및 추천을 진행하게 됨


---

### 📌 2.7 Top-N 추천 정확도 평가

💡 설명
- Top-N 추천 평가는 모델이 사용자에게 추천한 상위 N개의 아이템 중 실제로 사용자가 좋아한 아이템의 비율(Precision)과, 사용자가 좋아한 아이템 중에서 추천된 아이템의 비율(Recall)을 측정
- 추천 시스템에서 단순 RMSE 외에도 Precision/Recall 평가를 통해 사용자 만족도를 반영한 정성적 성능 평가가 중요함
- Top-5 추천 기준으로 Precision과 Recall을 평균하여 모델의 전반적인 추천 품질을 평가함

In [69]:
top_n = 5
threshold = 1.0  # 좋아요 간주 기준

precision_list = []
recall_list = []

for user_idx in filtered_ratings['user_idx'].unique()[:50]:  # 50명 테스트
    user_items_row = user_item_matrix[user_idx:user_idx+1]

    try:
        # 필터링 OFF → 범위 초과 방지
        recommended_items_scores = als_model.recommend(userid=user_idx, user_items=user_items_row, N=top_n, filter_already_liked_items=False)
    except Exception as e:
        print(f"User {user_idx}: 추천 에러 발생 → {e}")
        continue

    # recommended_items_scores는 (N, 2) 배열 → unpack X
    recommended_items = [int(item_score[0]) for item_score in recommended_items_scores]

    actual_liked_items = filtered_ratings[
        (filtered_ratings['user_idx'] == user_idx) &
        (filtered_ratings['rating'] >= threshold)
    ]['item_idx'].tolist()

    # print(f"User {user_idx}: 추천 {len(recommended_items)}개, 실제 좋아한 {len(actual_liked_items)}개")

    if len(actual_liked_items) == 0:
        print(f"User {user_idx}: 좋아한 항목 없음 → 평가 제외")
        continue

    y_true = [1 if item in actual_liked_items else 0 for item in recommended_items]
    y_pred = [1] * len(recommended_items)

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)

    precision_list.append(precision)
    recall_list.append(recall)

# 평균 Precision/Recall
avg_precision = np.mean(precision_list)
avg_recall = np.mean(recall_list)

print(f"\n✅ Top-{top_n} Precision: {avg_precision:.4f}")
print(f"✅ Top-{top_n} Recall: {avg_recall:.4f}")


✅ Top-5 Precision: 0.0400
✅ Top-5 Recall: 0.0800


🔍 결과
- Top-5 Precision은 약 4%, Recall은 약 0.8% 
- 이는 모델이 추천한 상위 5개 아이템 중 사용자가 실제로 좋아한 비율이 낮고, 사용자가 좋아한 아이템을 거의 추천하지 못했다는 것을 의미

낮은 Precision/Recall 값은 모델의 추천 품질이 부족함을 확인

<개선 방안>>
1. 하이퍼파라미터 튜닝: factors 수, regularization 값, iteration 수 조정
2. 추천 전략 개선: filter_already_liked_items 옵션을 활용하여 사용자 만족도 반영
3. 데이터 전처리 강화: 평점 정규화 또는 이상치 제거로 데이터 품질 향상
4. 다른 모델 시도: SVD, Autoencoder 기반 딥러닝 모델 등 명시적 피드백에 적합한 알고리즘 비교